In [1]:
import pandas as pd
import numpy as np
from config import DATA_PATH, FEATURES, TARGET
from features import FEATURE_DICT
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
df =pd.read_csv(DATA_PATH)
df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
N = 10
mu, sigma = 0, 1 # mean and standard deviation
df = pd.DataFrame({'col1': [np.random.normal(mu, sigma) for _ in range(N)], 
                   'col2': [np.random.normal(mu, sigma) for _ in range(N)], 
                   'col3': [np.random.normal(mu, sigma) for _ in range(N)], 
                   'col4': [np.random.normal(mu, sigma) for _ in range(N)], 
                   'col5': [np.random.normal(mu, sigma) for _ in range(N)], 
                   'col6': np.random.poisson(5, N),
                   'target': np.random.binomial(1, 0.3, N)})
df.head(5)

,col1,col2,col3,col4,col5,col6,target
0,-0.234159,-0.746435,1.089858,-0.175152,-0.029445,7,0
1,-1.071874,-0.582657,0.246168,0.157802,0.863291,4,0
2,0.301360,-1.486419,-1.338920,1.147685,0.405548,5,0
3,1.688116,0.721142,0.206520,-0.133091,1.199453,5,0
4,-0.443632,0.674544,-0.823113,0.727600,0.338874,7,1


In [4]:
#x_df = sm.add_constant(df)
df['constant'] = 1
df

,col1,col2,col3,col4,col5,col6,target,constant
0,-0.234159,-0.746435,1.089858,-0.175152,-0.029445,7,0,1
1,-1.071874,-0.582657,0.246168,0.157802,0.863291,4,0,1
2,0.301360,-1.486419,-1.338920,1.147685,0.405548,5,0,1
3,1.688116,0.721142,0.206520,-0.133091,1.199453,5,0,1
4,-0.443632,0.674544,-0.823113,0.727600,0.338874,7,1,1
5,0.209726,2.436334,-0.524083,1.883850,-0.173695,4,0,1
6,-1.336351,1.406393,0.190890,0.226507,0.079805,3,1,1
7,0.374980,-1.097211,-2.045208,1.035017,-0.976868,7,1,1
8,-1.084465,1.182664,-0.410976,1.333730,0.600288,3,0,1
9,0.692598,0.108555,0.152735,0.273260,0.345269,2,1,1


In [5]:
~df.columns.isin(['target'])

array([ True,  True,  True,  True,  True,  True, False,  True])

In [6]:
df.columns[~df.columns.isin(['target'])]

Index(['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'constant'], dtype='object')

In [7]:
model = sm.OLS(df.target, df[df.columns[~df.columns.isin(['target'])]])

In [8]:
results = model.fit()

In [9]:
results.params

col1       -0.095007
col2        0.240925
col3       -0.654113
col4       -1.004915
col5       -0.523084
col6       -0.098063
constant    1.365930
dtype: float64

In [10]:
results.summary()

C:\dev\virtual\gcp_prep\lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.276
Method:                 Least Squares   F-statistic:                     1.571
Date:                Tue, 06 Sep 2022   Prob (F-statistic):              0.382
Time:                        23:32:45   Log-Likelihood:               0.052903
No. Observations:                  10   AIC:                             13.89
Df Residuals:                       3   BIC:                             16.01
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
col1          -0.0950      0.167     -0.569      0.609      -0.626       0.436
col2           0.2409      0.162      1.488      0.233      -0.274       0.756
col3          -0.6541      0.300     -2.178      0.118      -1.610       0.302
col4          -1.0049      0.397     -2.528      0.086      -2.270       0.260
col5          -0.5231      0.304     -1.722      0.184      -1.490       0.444
col6          -0.0981      0.099     -0.994      0.393      -0.412       0.216
constant       1.3659      0.590      2.316      0.103      -0.511       3.243
==============================================================================
Omnibus:                        4.282   Durbin-Watson:                   2.189
Prob(Omnibus):                  0.118   Jarque-Bera (JB):                2.174
Skew:                           1.139   Prob(JB):                        0.337
Kurtosis:                       2.843   Cond. No.                         24.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Read your p-value first. If the p-value is small (less than your alpha level), you can reject the null hypothesis. Only then should you consider the f-value. If you don’t reject the null, ignore the f-value.

if you want to know whether your regression F-value is significant, ind the critical value in the f-table. For example, let’s say you had 3 regression degrees of freedom (df1) and 120 residual degrees of freedom (df2). An F statistic of at least 3.95 is needed to reject the null hypothesis at an alpha level of 0.1. At this level, you stand a 1% chance of being wrong. F Values will range from 0 to an arbitrarily large number.

The Akaike information criterion (AIC) is an estimator of prediction error and thereby relative quality of statistical models for a given set of data.

In estimating the amount of information lost by a model, AIC deals with the trade-off between the goodness of fit of the model and the simplicity of the model. In other words, AIC deals with both the risk of overfitting and the risk of underfitting.

As an example, suppose that there are three candidate models, whose AIC values are 100, 102, and 110. Then the second model is exp((100 − 102)/2) = 0.368 times as probable as the first model to minimize the information loss. Similarly, the third model is exp((100 − 110)/2) = 0.007 times as probable as the first model to minimize the information loss.

In this example, we would omit the third model from further consideration

Like skewness, kurtosis describes the shape of a probability distribution 

D: Laplace distribution, also known as the double exponential distribution, red curve (two straight lines in the log-scale plot), excess kurtosis = 3
S: hyperbolic secant distribution, orange curve, excess kurtosis = 2
L: logistic distribution, green curve, excess kurtosis = 1.2
N: normal distribution, black curve (inverted parabola in the log-scale plot), excess kurtosis = 0
C: raised cosine distribution, cyan curve, excess kurtosis = −0.593762...
W: Wigner semicircle distribution, blue curve, excess kurtosis = −1
U: uniform distribution, magenta curve (shown for clarity as a rectangle in both images), excess kurtosis = −1.2.

In [11]:
def RMSE(estimates, actual):
    """Computes the root mean squared error of a sequence of estimates.

    estimate: sequence of numbers
    actual: actual value

    returns: float RMSE
    """
    e2 = [(estimate-actual)**2 for estimate in estimates]
    mse = np.mean(e2)
    return np.sqrt(mse)

In [12]:
model = LogisticRegression()

In [13]:
model.fit(x_values, df.target)

NameError: name 'x_values' is not defined

In [ ]:
y_predict = model.predict(x_values)

In [ ]:
y_predict

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_estimator(
    model, x_values , df.target, name="LinearSVC"
)
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
from sklearn.metrics import (
    recall_score,
    precision_score,
    f1_score,
    average_precision_score,
    make_scorer,
)
for metric in [precision_score, recall_score, f1_score]:
        print(f"{metric.__name__} + : {metric(df.target, y_predict):.2}")
